In [ ]:
import matplotlib.pyplot as plt
import GridWorld
import numpy as np
import pandas as pd

In [ ]:
robot=GridWorld.grid_world()

In [ ]:
def print_values(V, rows,columns):
    for i in range(rows):
        print("---------------------------")
        for j in range(columns):
            v = V.get((i,j), 0)
            if v >= 0:
                print(" %.2f|" % v, end="")
            else:
                print("%.2f|" % v, end="") # -ve sign takes up an extra space
        print("")
def print_policy(policy,rows,columns):
    for i in range(rows):
        print("---------------------------")
        for j in range(columns):
            a = policy.get((i,j), ' ')
            print("  %s  |" % a, end="")
        print("")

In [ ]:
def max_dict(d):
    # returns the argmax (key) and max (value) from a dictionary
    # put this into a function since we are using it so often

    # find max val
    max_val = max(d.values())

    # find keys corresponding to max val
    max_keys = [key for key, val in d.items() if val == max_val]

    return np.random.choice(max_keys), max_val

In [ ]:
def epsilon_greedy(Q, s, eps=0.1):
    if np.random.random() < eps:
        return np.random.choice(robot.ACTION_SPACE)
    else:
        a_opt = max_dict(Q[s])[0]
    return a_opt

In [ ]:
# initialize Q(s,a) and returns
Q = {}
sample_counts = {}
for s in robot.States:
    Q[s] = {}
    sample_counts[s] = {}
    for a in robot.ACTION_SPACE:
        Q[s][a] = 0

In [ ]:
GAMMA=0.9
ALPHA = 0.1
update_counts = {}
# repeat until convergence
reward_per_episode = []
for it in range(10000):
    if it % 1000 == 0:
        print("it:", it)
    # begin a new episode
    s=robot.initial_state()
#    print(s)
    episode_reward = 0
    done=False
    while not done:
        # perform action and get next state + reward
        a = epsilon_greedy(Q, s, eps=0.1)
        s2, r, done=robot.step(a)
#        print(r)
        # update reward
        episode_reward += r
        # update Q(s,a)
        maxQ = max_dict(Q[s2])[1]
        Q[s][a] = Q[s][a] + ALPHA*(r + GAMMA*maxQ - Q[s][a])

        # we would like to know how often Q(s) has been updated too
        update_counts[s] = update_counts.get(s,0) + 1

        # next state becomes current state
        s = s2

        # log the reward for this episode
        reward_per_episode.append(episode_reward)

plt.plot(reward_per_episode)
plt.title("reward_per_episode")
plt.show()

In [ ]:
# determine the policy from Q*
# find V* from Q*
policy = {}
V = {}
for s in robot.actions.keys():
    a, max_q = max_dict(Q[s])
    policy[s] = a
    V[s] = max_q

In [ ]:
print_policy(policy,3,4)

In [ ]:
print_values(V,3,4)

In [ ]:
update_counts

In [ ]:
state_sample_count_arr = np.zeros((3, 4))
for i in range(3):
    for j in range(4):
        if (i, j) in update_counts:
            state_sample_count_arr[i,j] = update_counts[(i, j)]
    df = pd.DataFrame(state_sample_count_arr)
print(df)